In [100]:
import pandas as pd
import re
from textblob import TextBlob
import spacy as spacy
import numpy as np
nlp = spacy.load("pl_core_news_lg")
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import ShuffleSplit

# Cleaning and importing data


In [108]:
df = pd.read_csv('C:/Users/konra/analiza.csv',sep='__',encoding='utf-8')
df

<ipython-input-108-4df8a0abd9e6>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('C:/Users/konra/analiza.csv',sep='__',encoding='utf-8')


,Text,label,label;;;;;;
0,"Taksowak przyjezdza na czas , wydaje sie , ze ...",label,z_zero;;;;;;
1,Do tej pani trafili śmy z naszą 2 - miesięczna...,label,z_zero;;;;;;
2,"Wyróżnikiem , a także ciekawym elementem całeg...",label,z_zero;;;;;;
3,"W zależności od pory dnia , w naszych organizm...",label,z_zero;;;;;;
4,"Polecam wszystkim , zwłaszcza tym , dla któryc...",label,z_plus_m;;;;;;
...,...,...,...
45959,Na plus zaliczył by m też wbudowane gniazda US...,label,z_plus_m;;;;;;
45960,Bardzo blisko ( 3 - 5 min . drogi ) do centrum...,label,z_plus_m;;;;;;
45961,"Fachowo , prawie bezboleśnie i skutecznie , bo...",label,z_plus_m;;;;;;
45962,Podsumowując : 1 . umówił em się na prywatną w...,label,z_minus_m;;;;;;


In [109]:
def getRidOff(text):
    text=re.sub(r';','',text)
    text=re.sub(r'"','',text)
    return text
df['label;;;;;;']=df['label;;;;;;'].apply(getRidOff)

In [110]:
def cleanText(text):
    text=re.sub(r':','',text)
    text=re.sub(r';','',text)
    text=re.sub(r'[0-9]+','',text)
    text=re.sub(r'  ',' ',text)
    text=re.sub(r'#','',text)
    text=re.sub(r'-','',text)
    text=re.sub(r',','',text)
    text=re.sub(r'"','',text)
    return text
df['Text']=df['Text'].apply(cleanText)

In [111]:
df.drop('label',axis=1,inplace=True)

In [112]:
df=df.rename(columns={'label;;;;;;':'label'})

In [113]:
df.head(100)

,Text,label
0,Taksowak przyjezdza na czas wydaje sie ze je...,z_zero
1,Do tej pani trafili śmy z naszą miesięczna có...,z_zero
2,Wyróżnikiem a także ciekawym elementem całego...,z_zero
3,W zależności od pory dnia w naszych organizma...,z_zero
4,Polecam wszystkim zwłaszcza tym dla których ...,z_plus_m
...,...,...
95,Pan doktor miał do mnie strasznie szybkie po...,z_amb
96,Mam wrażenie że wręcz mu przeszkadzam !,z_minus_m
97,Obsługa uśmiechnięta i służy pomocą w zorganiz...,z_plus_m
98,Jednak cała otoczka związana z wizytami przypr...,z_minus_m


# Getting vector value of each sentence


In [117]:
def getVector(x):
    doc = nlp(x)
    vec = doc.vector
    return vec

In [118]:
df['Vector']=df['Text'].apply(lambda x: getVector(x))

In [119]:
df

,Text,label,Vector
0,Taksowak przyjezdza na czas wydaje sie ze je...,z_zero,"[-0.34779602, -0.15112223, 0.006167796, 0.1462..."
1,Do tej pani trafili śmy z naszą miesięczna có...,z_zero,"[-0.94308925, -2.1417625, 0.21697687, -0.69597..."
2,Wyróżnikiem a także ciekawym elementem całego...,z_zero,"[-0.27711576, -2.4430223, -0.5780529, -0.19264..."
3,W zależności od pory dnia w naszych organizma...,z_zero,"[-0.45596892, -1.0282898, -0.92031807, 0.11715..."
4,Polecam wszystkim zwłaszcza tym dla których ...,z_plus_m,"[0.30919805, -0.61286086, -0.90543896, -0.1594..."
...,...,...,...
45959,Na plus zaliczył by m też wbudowane gniazda US...,z_plus_m,"[-1.5377746, 0.35446644, -0.5560248, -1.080410..."
45960,Bardzo blisko ( min . drogi ) do centrum i ma...,z_plus_m,"[-0.7302746, 0.25571844, -1.2975017, -1.089486..."
45961,Fachowo prawie bezboleśnie i skutecznie bo e...,z_plus_m,"[-0.6959234, -0.015844243, -1.4620479, -0.2142..."
45962,Podsumowując . umówił em się na prywatną wizy...,z_minus_m,"[-0.80375636, -0.40092832, -0.5400778, -0.8969..."


# Splitting data to training and test sets

In [120]:
X = df['Vector'].to_numpy()
X=X.reshape(-1,1)

In [121]:
X.shape

(45964, 1)

In [122]:
X=np.concatenate(np.concatenate(X,axis=0),axis=0).reshape(-1,300)

In [123]:
X.shape

(45964, 300)

In [124]:
y=df['label']


In [125]:
y

0           z_zero
1           z_zero
2           z_zero
3           z_zero
4         z_plus_m
           ...    
45959     z_plus_m
45960     z_plus_m
45961     z_plus_m
45962    z_minus_m
45963        z_amb
Name: label, Length: 45964, dtype: object

In [126]:
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [127]:
X_train.shape,X_test.shape

((36771, 300), (9193, 300))

# Training Model


In [128]:
clf=LogisticRegression(solver='liblinear')

In [129]:
clf.fit(X_train, Y_train)

LogisticRegression(solver='liblinear')

In [132]:
Y_pred=clf.predict(X_test)

In [133]:
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

       z_amb       0.55      0.16      0.25      1138
   z_minus_m       0.63      0.78      0.70      3353
    z_plus_m       0.67      0.69      0.68      2426
      z_zero       0.70      0.70      0.70      2276

    accuracy                           0.66      9193
   macro avg       0.64      0.58      0.58      9193
weighted avg       0.65      0.66      0.64      9193



In [134]:
import pickle

In [135]:
pickle.dump(clf,open('w2v_Sentiment','wb'))

In [136]:
df=df[df["Text"].str.contains('youtube.com')==False]

In [143]:
x='Super tego właśnie potrzebowaliśmy te zmiany pomogą Polakom!'
x = cleanText(x)

In [144]:
vector=getVector(x)

In [145]:
clf.predict(vector.reshape(1,-1))

array(['z_plus_m'], dtype=object)